# Learn Dinosaur Name
Get 3 letters, predict the next letter

In [0]:
import tensorflow.keras as keras
from tensorflow.keras.layers import *
from tensorflow.keras.utils import to_categorical

In [2]:
from google.colab import files
uploaded = files.upload()   # one dinosaur name per line

In [3]:
names = []
with open('dinosaur.txt') as f:
    for line in f:
        names.append(line.lower().strip() + '{')   # open bracelet { is used as a string ending
        
print(names[0])        
len(names)

aachenosaurus{


1325

In [4]:
X = []
Y = []
for s in names:
    for i in range(len(s) - 3):
        X.append(s[i:i+3])
        Y.append(s[i+3])
print(X[0])
len(X)

aac


12986

In [5]:
X = [[ord(i) - ord('a') for i in x] for x in X]
X[0]


[0, 0, 2]

In [6]:
X = to_categorical(X)
X[0]

array([[1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
        0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)

In [7]:
X.shape

(12986, 3, 26)

In [8]:
Y = [[ord(i) - ord('a') for i in y] for y in Y]
print(Y[0])
Y = to_categorical(Y)
Y.shape

[7]


(12986, 27)

# Create a Model

In [47]:
cell = 64
input_size = X.shape[2] # one hot
time_steps = X.shape[1] # seq len    

model = keras.Sequential([
        GRU(cell, activation='relu', input_shape=(time_steps, input_size)),
        #Dense(Y.shape[1] * 2, activation='relu'),
        Dense(Y.shape[1], activation='softmax'),
])
model.summary()

Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 64)                17472     
_________________________________________________________________
dense_10 (Dense)             (None, 54)                3510      
_________________________________________________________________
dense_11 (Dense)             (None, 27)                1485      
Total params: 22,467
Trainable params: 22,467
Non-trainable params: 0
_________________________________________________________________


In [39]:
'''
input_size = X.shape[2] # one hot
time_steps = X.shape[1] # seq len    

model = keras.Sequential([
        LSTM(64, return_sequences=True, activation='relu', input_shape=(time_steps, input_size)),  # stacked LSTM : return_sequences=True
        LSTM(32, activation='relu', ),
        Dense(Y.shape[1], activation='softmax', use_bias=False)
])
model.summary()
'''

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_8 (LSTM)                (None, 3, 64)             23296     
_________________________________________________________________
lstm_9 (LSTM)                (None, 32)                12416     
_________________________________________________________________
dense_7 (Dense)              (None, 27)                864       
Total params: 36,576
Trainable params: 36,576
Non-trainable params: 0
_________________________________________________________________


In [0]:
model.compile(optimizer=keras.optimizers.Adam(), 
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [49]:
model.fit(X, Y, epochs=30)

Epoch 1/30
12986/12986 [==============================] - 5s 351us/sample - loss: 2.2467 - acc: 0.3746
Epoch 2/30
12986/12986 [==============================] - 4s 316us/sample - loss: 1.7284 - acc: 0.4919
Epoch 3/30
12986/12986 [==============================] - 4s 312us/sample - loss: 1.6319 - acc: 0.5247
Epoch 4/30
12986/12986 [==============================] - 4s 317us/sample - loss: 1.5786 - acc: 0.5447
Epoch 5/30
12986/12986 [==============================] - 4s 317us/sample - loss: 1.5346 - acc: 0.5564
Epoch 6/30
12986/12986 [==============================] - 4s 313us/sample - loss: 1.5002 - acc: 0.5652
Epoch 7/30
12986/12986 [==============================] - 4s 314us/sample - loss: 1.4708 - acc: 0.5735
Epoch 8/30
12986/12986 [==============================] - 4s 313us/sample - loss: 1.4434 - acc: 0.5857
Epoch 9/30
12986/12986 [==============================] - 4s 313us/sample - loss: 1.4206 - acc: 0.5898
Epoch 10/30
12986/12986 [==============================] - 4s 311us/sampl

# Generate Cool names

In [0]:
def dinoname(s):
    name = s
    while True:
        x = [ord(c) - ord('a') for c in s]
        x = to_categorical(x, 26)
        x = x.reshape(1, 3, 26)
        c = model.predict_classes(x)
        c = chr(ord('a') + c)
        if c == '{':
            break
        else:
            name += c
            s = s[1:] + c
    return name

In [51]:
a = ['ant', 'bat', 'cat', 'dog', 'eel', 'fox', 'gnu', 'hog', 'imu', 'jay', 'owl', 'pig', 'rat', 'yak']
[dinoname(s).capitalize() for s in a]

['Anthosaurus',
 'Batitan',
 'Catitan',
 'Dognatops',
 'Eellisaurus',
 'Foxa',
 'Gnus',
 'Hognatops',
 'Imus',
 'Jayangosaurus',
 'Owlong',
 'Pigong',
 'Ratops',
 'Yakeria']